In [3]:
from pixell import enmap, enplot, utils, curvedsky
from cosmikyu import maps, config
from cosmikyu import utils as cutils
from soapack import interfaces as soint
import healpy as hp
import numpy as np
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
output_dir = config.default_data_dir
def output_path(x):
    return os.path.join(output_dir, x)

In [9]:
sehgal_sim_path = soint.dconfig['actsims']['sehgal09_path']
SH = maps.SehgalSky2010(sehgal_sim_path, data_type='alm')

freq_idxes = ['030', '090', '148', '219', '277', '350']
fg_loaders = {
    'ir_pts': SH.get_cib,
    #'ksz': SH.get_ksz,
    #'rad_pts': SH.get_radio,
    #'tsz': SH.get_tsz   
}

res = 0.5*utils.arcmin
fshape, fwcs = enmap.fullsky_geometry(res)
template = maps.get_template(fshape, fwcs)

for freq_idx in freq_idxes:
    for fg_idx in fg_loaders.keys():
        target_file = output_path('{}/{}_meta.txt'.format(fg_idx, freq_idx))
        if os.path.exists(target_file):
            print("skipping {} {}".format(freq_idx, fg_idx))
        else:
            print("processing {} {}".format(freq_idx, fg_idx))
            os.makedirs(output_path(fg_idx), exist_ok=True)
            
            alm = fg_loaders[fg_idx](freq_idx)
            fgmap = curvedsky.alm2map(alm, template.copy())
            meta = np.array([np.mean(fgmap), np.std(fgmap), np.max(fgmap), np.min(fgmap)])
            np.savetxt(target_file, meta, header="mean,std,max,min",
                       comments="#[{} F{} 0.5 arcmin res]   ".format(fg_idx.upper(), freq_idx))
            
            del alm, fgmap
            
#sign = np.sign(fgmap)
#fgmap = np.log(np.abs(fgmap))*sign
            
def load_meta(fg_idxes, freq_idxes):
    ret = cutils.create_dict(fg_idxes, freq_idxes)
    for freq_idx in freq_idxes:
        for fg_idx in fg_loaders.keys():
            target_file = output_path('{}/{}_meta.txt'.format(fg_idx, freq_idx)) 
            mmean, mstd, mmax, mmin = np.loadtxt(target_file)
            ret[fg_idx][freq_idx] = {'mean': mmean, 'std': mstd, 'max': mmax , 'min': mmin}
    return ret
    
metas = load_meta(list(fg_loaders.keys()), freq_idxes)
for fg_idx in fg_loaders.keys():
    for freq_idx in metas[fg_idx].keys():
        print(fg_idx, freq_idx, metas[fg_idx][freq_idx])
    


skipping 030 ir_pts
skipping 090 ir_pts
skipping 148 ir_pts
skipping 219 ir_pts
skipping 277 ir_pts
skipping 350 ir_pts
ir_pts 030 {'mean': 34.04549400407869, 'std': 29.18152531528908, 'max': 1962.4244099098073, 'min': -279.6274886596626}
ir_pts 090 {'mean': 1356.5323079135794, 'std': 1174.415770589677, 'max': 68714.10429188391, 'min': -9749.374288827685}
ir_pts 148 {'mean': 6723.738431885741, 'std': 5802.9026643761335, 'max': 348620.8448433348, 'min': -47906.45000015778}
ir_pts 219 {'mean': 21881.05272039636, 'std': 18976.60609857938, 'max': 1174320.0997806764, 'min': -186649.57752546904}
ir_pts 277 {'mean': 41986.01840417685, 'std': 37355.620383476635, 'max': 2435753.0602501566, 'min': -397872.74498460983}
ir_pts 350 {'mean': 75860.88969795564, 'std': 68821.71759447258, 'max': 4540703.675758926, 'min': -755946.9776038778}


In [6]:
for freq_idx in freq_idxes:
    for fg_idx in fg_loaders.keys():
        alm = fg_loaders[fg_idx](freq_idx)()
        for sim_idx in np.arange(10000):
            target_file = output_path('{}/{}_{}_{}.fits'.format(fg_idx, freq_idx, '%06d'%sim_idx, res))

            if os.path.exists(target_file):
                continue
            else:
                if sim_idx %100 == 0:
                    print("generating {}".format(target_file))
                try:
                    shape, wcs, _ = maps.rand_geometry(stamp_width, stamp_height, shape=shape, seed=sim_idx)
                except:
                    pass

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [8]:
wcs, _, _ = maps.get_geometry(1.3520746650524709,1.7241484721289566 , stamp_width, stamp_width, res=0.5)

enmap.wcsutils.is_compatible(fwcs, fwcs)

True

In [20]:
type(wcs)

astropy.wcs.wcs.WCS

In [ ]:
maps.rand_geometry??

In [11]:
np.sign([-5,0,5])

array([-1,  0,  1])